In [ ]:
import h5py
import numpy as np

filename = "./clgs_results_final_float32.h5"
def get_parameters():
    with h5py.File(filename, 'r') as file:
        case = "{tube_shape}"
        fluid = "sCO2"
        input_loc = "/" + case + "/" + fluid + "/input/"
        mdot = file[input_loc + "mdot"][:]  # i0
        L2 = file[input_loc + "L2"][:]  # i1
        L1 = file[input_loc + "L1"][:]  # i2
        grad = file[input_loc + "grad"][:]  # i3
        D = file[input_loc + "D"][:]  # i4
        Tinj = file[input_loc + "T_i"][:]  # i5
        k = file[input_loc + "k_rock"][:]  # i6
        time = file[input_loc + "time"][:]  # i7
        return [mdot, L2, L1, grad, D, Tinj, k, time]

parameter_values = get_parameters()
parameter_values

[array([  5. ,   8.8,  12.6,  16.4,  20.2,  24. ,  27.8,  31.6,  35.4,
         39.2,  43. ,  46.8,  50.6,  54.4,  58.2,  62. ,  65.8,  69.6,
         73.4,  77.2,  81. ,  84.8,  88.6,  92.4,  96.2, 100. ],
       dtype=float32),
 array([ 1000.,  2000.,  3000.,  4000.,  5000.,  6000.,  7000.,  8000.,
         9000., 10000., 11000., 12000., 13000., 14000., 15000., 16000.,
        17000., 18000., 19000., 20000.], dtype=float32),
 array([1000., 1500., 2000., 2500., 3000., 3500., 4000., 4500., 5000.],
       dtype=float32),
 array([0.03, 0.04, 0.05, 0.06, 0.07], dtype=float32),
 array([0.2159, 0.3302, 0.4445], dtype=float32),
 array([303.15, 318.15, 333.15], dtype=float32),
 array([1.5, 3. , 4.5], dtype=float32),
 array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,
         2.25,  2.5 ,  2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.25,
         4.5 ,  4.75,  5.  ,  5.25,  5.5 ,  5.75,  6.  ,  6.25,  6.5 ,
         6.75,  7.  ,  7.25,  7.5 ,  7.75,  8.  ,  8.25,  8.5 ,  8.7

In [1]:
import zarr
import h5py
import numpy as np

filename = "./clgs_results_final_float32.h5"
shape = (26, 20, 9, 5, 3, 3, 3, 161)
shapes = ["utube", "coaxial"]
fluids = ["H2O", "sCO2"]
types = ["Tout", "Pout"]
for tube_shape in shapes:
    for fluid in fluids:
        for type in types:
            with h5py.File(filename, 'r') as file:
                We = file[f"/{tube_shape}/{fluid}/output/We"][:]
                U = file[f"/{tube_shape}/{fluid}/output/{type}" + "/" + "U"][:]
                sigma = file[f"/{tube_shape}/{fluid}/output/{type}" + "/" + "sigma"][:]
                Vt = file[f"/{tube_shape}/{fluid}/output/{type}" + "/" + "Vt"][:]

                M_k = np.dot(U, np.dot(np.diag(sigma), Vt))

                valid_runs = np.argwhere(np.isfinite(We.flatten()))[:, 0]
                M_k_full = np.full((shape[-1], np.prod(shape[:-1])), np.nan)
                M_k_full[:, valid_runs] = M_k
                ans = np.reshape(M_k_full.T, shape)
                ans = ans.astype(np.float32)
                print(ans.shape)
                
                chunk_size = (26, 1, 1, 1, 1, 1, 1, 161)
                z = zarr.creation.array(ans, chunks=chunk_size, store=f'{tube_shape}_{fluid}_{type}.zarr', overwrite=True)

(26, 20, 9, 5, 3, 3, 3, 161)
(26, 20, 9, 5, 3, 3, 3, 161)
(26, 20, 9, 5, 3, 3, 3, 161)
(26, 20, 9, 5, 3, 3, 3, 161)
(26, 20, 9, 5, 3, 3, 3, 161)
(26, 20, 9, 5, 3, 3, 3, 161)
(26, 20, 9, 5, 3, 3, 3, 161)
(26, 20, 9, 5, 3, 3, 3, 161)
